In [1]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [2]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [3]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 30
n_iter = 13000 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [4]:
import numpy as np
import impl.loss as loss_fun
import impl.layer as l
import impl.regularization as reg
import impl.utils as util
import impl.NN as nn

class RNN(nn.NN):

    def __init__(self, D, H, char2idx, idx2char):
        self.D = D
        self.H = H
        self.char2idx = char2idx
        self.idx2char = idx2char
        self.vocab_size = len(char2idx)
        super().__init__(D, D, H, None, None, loss='cross_ent', nonlin='relu')

    def initial_state(self):
        return np.zeros((1, self.H))

    def forward(self, X, h, num_layer, train=True):
        model = self.models[num_layer-1]
        Wxh, Whh, Why = model['Wxh'], model['Whh'], model['Why']
        bh, by = model['bh'], model['by']

#         if num_layer==1: # if the first layer, the input should be integers
#             X_one_hot = np.zeros(self.D)
#             X_one_hot[X] = 1.
#         else: X_one_hot = X      
#         X_one_hot = X_one_hot.reshape(1, -1)
        X_one_hot = np.zeros(self.D)
        X_one_hot[X] = 1.
        X_one_hot = X_one_hot.reshape(1, -1)

        hprev = h.copy()

        h, h_cache = l.tanh_forward(X_one_hot @ Wxh + hprev @ Whh + bh)
        y, y_cache = l.fc_forward(h, Why, by)

        cache = (X_one_hot, Whh, h, hprev, y, h_cache, y_cache, Wxh)

        if not train:
            y = util.softmax(y)

        return y, h, cache

    def backward(self, dy, dh_next, cache):
        X, Whh, h, hprev, y, h_cache, y_cache, Wxh = cache

        # Hidden to output gradient
        dh, dWhy, dby = l.fc_backward(dy, y_cache)
        dh += dh_next
        dby = dby.reshape((1, -1))

        # tanh
        dh = l.tanh_backward(dh, h_cache)

        # Hidden gradient
        dbh = dh
        dWhh = hprev.T @ dh
        dWxh = X.T @ dh
        dh_next = dh @ Whh.T
        dX = dh @ Wxh.T

        grad = dict(Wxh=dWxh, Whh=dWhh, Why=dWhy, bh=dbh, by=dby)

        return grad, dh_next, dX

    def train_step_fwd(self, X_train, y_train, h):

        # For all the layers
        ys_, caches_ = [], []

        # Forward 1st layer
        ys, caches = [], []
        h_ = h.copy()
        for x in X_train:
            y_pred, h_, cache = self.forward(x, h_, train=True, num_layer=1) # self.model[0] for the 1st layer
            ys.append(y_pred)
            caches.append(cache)
        ys_.append(ys)
        caches_.append(caches)

#         # Forward 2nd layer
#         ys, caches = [], []
#         h_ = h.copy()
#         for x in ys_[0]: # the output of the 1st layer as the input for the 2nd layer
#             y_pred, h_, cache = self.forward(x, h_, train=True, num_layer=2) # self.model[1] for the 2nd layer
#             ys.append(y_pred)
#             caches.append(cache)
#         ys_.append(ys)
#         caches_.append(caches)
            
        return ys_, caches_

    def train_step_bwd(self, X_train, y_train, ys_, caches_):

        # loss/error function: last layer==1st layer
        loss, dys, grads_ = 0.0, [], []
        for y_pred, y in zip(ys_[0], y_train):
            loss += loss_fun.cross_entropy(self.models[0], y_pred, y, lam=0)/ y_train.shape[0] # last layer
            dy = loss_fun.dcross_entropy(y_pred, y)
            dys.append(dy)
            
#         # Backward 2nd layer
#         dh_next = np.zeros((1, self.H))
#         grads = {k: np.zeros_like(v) for k, v in self.models[1].items()} # for the 2nd layer
#         dys_ = dys.copy()
#         dys = []

#         for t in reversed(range(len(dys_))):
#             grad, dh_next, dX = self.backward(dys_[t], dh_next, caches_[1][t]) # self.model[1] for the 2nd layer
#             for k in grads.keys():
#                 grads[k] += grad[k]
#             dys.append(dX)
#         grads_.append(grads)

        # Backward 1st layer
        dh_next = np.zeros((1, self.H))
        grads = {k: np.zeros_like(v) for k, v in self.models[0].items()} # for the 1st layer
        dys_ = dys.copy()
        dys = []

        for t in reversed(range(len(dys_))):
            grad, dh_next, dX = self.backward(dys_[t], dh_next, caches_[0][t]) # self.model[0] for the 1st layer
            for k in grads.keys():
                grads[k] += grad[k]
            dys.append(dX) # as the input for previous layer
        grads_.append(grads)

        #         for grads in grads_:
        #             # Clipping grads for exploding grad problems
        #             for k, v in grads.items():
        #                 grads[k] = np.clip(v, -5., 5.)

        return grads_, loss

    def _init_model(self, D, C, H):
        self.models = []
        total_num_layers = 1
        for l in range(total_num_layers):
            model = dict(
                Wxh=np.random.randn(D, H) / np.sqrt(D / 2.),
                Whh=np.random.randn(H, H) / np.sqrt(H / 2.),
                Why=np.random.randn(H, D) / np.sqrt(C / 2.),
                bh=np.zeros((1, H)),
                by=np.zeros((1, D))
            )
            self.models.append(model)

In [5]:
net = RNN(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [6]:
import numpy as np
import impl.utils as util
import impl.constant as c
import copy
from sklearn.utils import shuffle as skshuffle

def get_minibatch(X, y, minibatch_size, shuffle=True):
    minibatches = []

    if shuffle:
        X, y = skshuffle(X, y)

    for i in range(0, X.shape[0], minibatch_size):
        X_mini = X[i:i + minibatch_size]
        y_mini = y[i:i + minibatch_size]

        minibatches.append((X_mini, y_mini))

    return minibatches

def adam_rnn(nn, X_train, y_train, alpha=0.001, mb_size=256, n_iter=2000, print_after=100):
    minibatches = get_minibatch(X_train, y_train, mb_size, shuffle=False)

    idx = 0
    state = nn.initial_state()
    smooth_loss = -np.log(1.0 / len(set(X_train)))

    M = {k: np.zeros_like(v) for k, v in nn.models[0].items()}
    R = {k: np.zeros_like(v) for k, v in nn.models[0].items()}
    beta1 = .9
    beta2 = .999

    for iter in range(1, n_iter + 1):
        t = iter

        if idx >= len(minibatches):
            idx = 0
            state = nn.initial_state()

        X_mini, y_mini = minibatches[idx]
        idx += 1

        if iter % print_after == 0:
            print('Iter-{} loss: {:.4f}'.format(iter, smooth_loss))

        #             # Testing can be completed once we make sure the training is done and is validated.
        #             sample = nn.test_step_fwd(X_mini[0], state)
        #             print(sample)

        ys_, caches_ = nn.train_step_fwd(X_mini, y_mini, state)
        grads_, loss = nn.train_step_bwd(X_mini, y_mini, ys_, caches_)
        smooth_loss = 0.999 * smooth_loss + 0.001 * loss

        for model, grads in zip(nn.models, grads_):
            for k in grads:
                M[k] = util.exp_running_avg(M[k], grads[k], beta1)
                R[k] = util.exp_running_avg(R[k], grads[k]**2, beta2)

                m_k_hat = M[k] / (1. - beta1**(t))
                r_k_hat = R[k] / (1. - beta2**(t))

                model[k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + c.eps)

    return nn

In [7]:
adam_rnn(nn=net, X_train=X, y_train=y, alpha=alpha, mb_size=time_step, n_iter=n_iter, 
                print_after=print_after)

Iter-1000 loss: 3.4149
Iter-2000 loss: 2.6713
Iter-3000 loss: 2.1698
Iter-4000 loss: 1.8278
Iter-5000 loss: 1.6015
Iter-6000 loss: 1.4451
Iter-7000 loss: 1.3216
Iter-8000 loss: 1.2236
Iter-9000 loss: 1.1423
Iter-10000 loss: 1.0817
Iter-11000 loss: 1.0124
Iter-12000 loss: 0.9530
Iter-13000 loss: 0.8942
